# 연구 및 기사 작성을 위한 에이전트 생성하기

멀티 에이전트 시스템(multi-agent systems)의 기본 개념을 소개하고, CrewAI 프레임워크에 대한 개요를 제공합니다.

In [1]:
# pip install crewai crewai-tools
# %pip install 'crewai[tools]'

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from crew_ai_utils import pretty_print_result, get_openai_api_key

import dotenv

dotenv.load_dotenv()
openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")


## 에이전트 생성
- 에이전트를 정의하고, `role` (역할), `goal` (목표) 및 `backstory` (배경 이야기) 정의
- LLM(Large Language Model)은 역할극을 할 때 성능이 더 좋은 것으로 알려져 있음

### Agent: Planner(콘텐츠 기획자)


In [4]:
planner = Agent(
    role="콘텐츠 기획자 (Content Planner)",
    goal="매력적이고 사실에 근거한 콘텐츠를 {topic}(주제)에 대해 기획합니다.",
    backstory="당신은 {topic}(주제)에 대한 블로그 기사 기획 작업을 하고 있습니다."
              "당신은 독자들에게 유용한 정보를 수집하여 독자들이 학습하고 "
              "정보에 입각한 결정을 내릴 수 있도록 돕습니다. "
              "당신의 작업은 콘텐츠 작가(Content Writer)가 이 주제에 대한 기사를 작성하는 데 기초가 됩니다.",
    allow_delegation=True,
	verbose=True
)

### Agent: Writer(콘텐츠 작가)

In [5]:
writer = Agent(
    role="콘텐츠 작가 (Content Writer)",
    goal="주제({topic})에 대해 통찰력 있고 사실에 근거한 의견(opinion)을 작성합니다.",
    backstory="당신은 주제({topic})에 대한 새로운 의견(opinion)을 작성하고 있습니다. "
              "당신은 콘텐츠 기획자(Content Planner)의 작업, 즉 개요(outline)와 주제에 대한 관련 맥락(context)을 기반으로 글을 씁니다. "
              "당신은 콘텐츠 기획자(Content Planner)가 제공한 개요(outline)의 주요 목표와 방향을 따릅니다. "
              "또한 객관적이고 공정한 통찰력(insights)을 제공하고 콘텐츠 기획자(Content Planner)가 제공한 정보를 통해 이를 뒷받침합니다. "
              "당신은 당신의 진술이 객관적인 진술이 아닌 의견(opinions)일 때 의견(opinion)에서 인정합니다.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor(편집자)

In [6]:
editor = Agent(
    role="편집자 (Editor)",
    goal="제공된 블로그 게시물을 편집하여 조직의 글쓰기 스타일에 맞춥니다.",
    backstory="당신은 콘텐츠 작가(Content Writer)로부터 블로그 게시물을 받는 편집자입니다. "
              "당신의 목표는 블로그 게시물을 검토하여 저널리즘의 모범 사례를 따르고,"
              "의견이나 주장을 제시할 때 균형 잡힌 관점을 제공하고,"
              "가능한 경우 주요 논쟁 거리가 되는 주제나 의견을 피하는 것입니다.",
    allow_delegation=False,
    verbose=True
)

## 작업 생성

- `task`, `description`, `expected_output`, `agent` 정의

### Task: Plan(콘텐츠 계획)

In [7]:
plan = Task(
    description=(
        "1. {topic}에 대한 최신 트렌드(latest trends), 주요 업체(key players) 및 주목할 만한 뉴스(noteworthy news)를 우선적으로 고려합니다.\n"
        "2. 대상 독자(target audience)의 관심사(interests)와 고충(pain points)을 고려하여 파악합니다.\n"
        "3. 서론(introduction), 핵심 내용(key points), 행동 촉구(call to action)를 포함한 상세한 콘텐츠 개요(content outline)를 작성합니다.\n"
        "4. SEO 키워드(SEO keywords)와 관련된 데이터(data) 또는 소스(sources)를 포함합니다.\n"
    ),
    expected_output="개요(outline), 대상 독자 분석(audience analysis), "
        "SEO 키워드(SEO keywords) 및 리소스(resources)를 포함하는 포괄적인 콘텐츠 계획 문서(comprehensive content plan document)입니다.",
    agent=planner,
)

### Task: Write(콘텐츠 작성)

In [8]:
write = Task(
    description=(
        "1. 콘텐츠 계획(content plan)을 활용하여 {topic}에 대한 설득력 있는 블로그 게시물(blog post)을 작성합니다.\n"
        "2. SEO 키워드(SEO keywords)를 자연스럽게 통합합니다.\n"
        "3. 섹션/소제목(Sections/Subtitles)이 매력적인 방식으로 적절하게 명명되었는지 확인합니다.\n"
        "4. 게시물이 매력적인 서론(introduction), 통찰력 있는 본문(insightful body) 및 요약 결론(summarizing conclusion)으로 구성되었는지 확인합니다.\n"
        "5. 문법적 오류(grammatical errors)가 없는지, 브랜드의 목소리(brand's voice)와 일치하는지 교정합니다(Proofread).\n"
    ),
    expected_output="잘 작성된 블로그 게시물(blog post)을 마크다운 형식(markdown format)으로 제공하며, 각 섹션은 2~3개의 단락(paragraphs)으로 구성되어 출판 준비가 완료되어야 합니다.",
    agent=writer,
)

### Task: Edit(콘텐츠 편집)

In [9]:
edit = Task(
    description=(
        "1. 주어진 블로그 게시물(blog post)에서 문법적 오류(grammatical errors)를 교정하고 브랜드의 목소리(brand's voice)와 일치하는지 확인합니다.\n"
    ),
    expected_output=(
        "잘 작성된 블로그 게시물(blog post)을 마크다운 형식(markdown format)으로 제공하며, "
        "각 섹션은 2~3개의 단락(paragraphs)으로 구성되어 출판 준비가 완료되어야 합니다."
    ),
    agent=editor,
)

## Crew 구성

- 에이전트(Agents)로 Crew를 구성
- 해당 에이전트(agents)가 수행할 작업(tasks)을 전달
    - 이 간단한 예제에서는* 작업(tasks)이 순차적으로 수행 됨.(즉, 서로 종속적)  
    - 따라서 목록에서 작업(task)의 _순서(order)_ 가 중요함.
- `verbose=True`를 설정하면 실행에 대한 모든 로그를 볼 수 있음

In [10]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, edit, write],
    manager_llm=ChatOpenAI(model="gpt-4o", 
                           temperature=0.7),   
    process=Process.sequential,
    verbose=True
)

## Running the Crew

In [11]:
result = crew.kickoff(inputs={"topic": "인공지능"})

# Agent: 콘텐츠 기획자 (Content Planner)
## Task: 1. 인공지능에 대한 최신 트렌드(latest trends), 주요 업체(key players) 및 주목할 만한 뉴스(noteworthy news)를 우선적으로 고려합니다.
2. 대상 독자(target audience)의 관심사(interests)와 고충(pain points)을 고려하여 파악합니다.
3. 서론(introduction), 핵심 내용(key points), 행동 촉구(call to action)를 포함한 상세한 콘텐츠 개요(content outline)를 작성합니다.
4. SEO 키워드(SEO keywords)와 관련된 데이터(data) 또는 소스(sources)를 포함합니다.



# Agent: 콘텐츠 기획자 (Content Planner)
## Using tool: Ask question to coworker
## Tool Input: 
"{\"question\": \"What are the latest trends and key players in the field of artificial intelligence?\", \"context\": \"I am planning a blog article on AI and need to include information on the latest trends, key players, and noteworthy news in this field.\", \"coworker\": \"\\ucf58\\ud150\\uce20 \\uc791\\uac00\"}"
## Tool Output: 

Error executing tool. coworker mentioned not found, it must be one of the following options:
- 콘텐츠 작가 (content writer)
- 편집자 (editor)

# Agent: 콘텐츠 작가 (Content Writer)
## Task: What are the la

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result.raw)

```markdown
# Exploring the Evolution and Impact of Artificial Intelligence

## Introduction

In an era where digital technology rapidly advances, artificial intelligence (AI) has emerged as a pivotal force reshaping industries across the globe. This article delves into the current trends of AI, highlighting the significant players driving the revolution and examining the profound implications for both businesses and individuals.

As AI continues to evolve, understanding its trajectory and influence becomes imperative. We will explore the latest advancements in AI technologies, discuss the roles of prominent companies and startups in this sphere, and consider how these changes affect contemporary society.

## Key Points

### Latest Trends in AI

AI's landscape is varied and expansive, with several key trends surfacing in recent years. Generative AI is revolutionizing creative processes, while AI applications in healthcare are enhancing diagnostic accuracy and patient care. Ethical AI is gaining focus, emphasizing the need for transparency and fairness in AI systems. Autonomous systems, such as self-driving cars, are nearing widespread adoption, and AI's integration with the Internet of Things (IoT) is creating smarter environments. Noteworthy technologies like OpenAI's GPT models, along with contributions from Google, IBM, and Tesla, are at the forefront of these innovations.

### Key Players in the AI Landscape

In the competitive world of AI, companies like Google, Microsoft, IBM, and Amazon lead the charge, pushing the boundaries of what's possible. These industry giants are joined by nimble startups offering novel solutions and challenging traditional paradigms. Together, they shape the AI ecosystem, fostering an environment ripe for innovation and growth.

### Target Audience Interests and Challenges

To demystify AI, it is crucial for audiences to grasp its basic concepts and practical applications. Advancements bring with them ethical concerns and regulatory questions that demand attention. Implementation strategies present challenges such as complexity and integration issues, while the field faces a notable talent shortage. Addressing these pain points, solutions involve simplifying integration processes, prioritizing ethical AI practices, and emphasizing cost-effective strategies.

### Noteworthy News and Developments

AI's swift evolution is reflected in recent breakthroughs reported by trusted tech sources and research publications. From advancements in natural language processing to strides in autonomous technology, keeping abreast of these developments is essential for anyone engaging with AI in their industry or academic pursuit.

## Call to Action

Staying informed is key to navigating the dynamic realm of AI. Readers are encouraged to continually engage with emerging trends and insights regarding AI's role in the future. Exploring online courses or delving into research papers can provide deeper understanding and skills, empowering individuals to harness the full potential of AI.

Consider posing questions to provoke thought and drive discussions within your networks: How will AI shape the workforce of the future? What ethical considerations must we prioritize as AI technologies become more pervasive?

By actively participating in the AI dialogue, readers can equip themselves to not only adapt to, but also shape, an AI-driven world.

---

This article provides a comprehensive look at artificial intelligence today, underlining the trends, players, and challenges that define its ever-evolving landscape. It aims to equip its audience—from tech enthusiasts to business leaders—with the insights necessary to succeed in an AI-augmented future.
```

This blog post should be well-suited for publication, aligning with the brand's voice and meeting organizational standards.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [15]:
topic = "2025년 3월 2일 서울 날씨"
result = crew.kickoff(inputs={"topic": topic})

# Agent: 콘텐츠 작가 (Content Writer)
## Task: 1. 콘텐츠 계획(content plan)을 활용하여 2025년 3월 2일 서울 날씨에 대한 설득력 있는 블로그 게시물(blog post)을 작성합니다.
2. SEO 키워드(SEO keywords)를 자연스럽게 통합합니다.
3. 섹션/소제목(Sections/Subtitles)이 매력적인 방식으로 적절하게 명명되었는지 확인합니다.
4. 게시물이 매력적인 서론(introduction), 통찰력 있는 본문(insightful body) 및 요약 결론(summarizing conclusion)으로 구성되었는지 확인합니다.
5. 문법적 오류(grammatical errors)가 없는지, 브랜드의 목소리(brand's voice)와 일치하는지 교정합니다(Proofread).

# Agent: 콘텐츠 기획자 (Content Planner)
## Task: What are the SEO keywords we need to integrate naturally into the blog post about 2025년 3월 2일 서울 날씨, and can you provide more details on the brand's voice for consistency?


# Agent: 콘텐츠 기획자 (Content Planner)
## Final Answer: 
To effectively plan for the blog post about the weather in Seoul on March 2, 2025, incorporating relevant SEO keywords is crucial for maximizing search engine visibility. Here are some suggested keywords to integrate naturally into the content:

1. **2025 Seoul Weather**
2. **March 2 Weather Forecast Seoul**


In [17]:
from IPython.display import Markdown
Markdown(result.raw)

The comprehensive content plan for the 2025년 3월 2일 서울 날씨 blog post includes a detailed outline (introduction, key points, conclusion, call to action), an audience analysis outlining interests and pain points, a list of SEO keywords, and suggested resources for information and inspiration.